In [1]:
import findspark
findspark.init('/home/ubuntu/spark-3.3.0-bin-hadoop3')
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('linear_regression').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/08 13:23:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
from pyspark.ml.regression import LinearRegression

In [3]:
traning = spark.read.format('libsvm').load('sample_linear_regression_data.txt')

22/09/08 13:23:58 WARN LibSVMFileFormat: 'numFeatures' option not specified, determining the number of features by going though the input. If you know the number in advance, please specify it via 'numFeatures' option to avoid the extra scan.


In [4]:
traning.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
| -7.896274316726144|(10,[0,1,2,3,4,5,...|
| -8.464803554195287|(10,[0,1,2,3,4,5,...|
| 2.1214592666251364|(10,[0,1,2,3,4,5,...|
| 1.0720117616524107|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
| -5.082010756207233|(10,[0,1,2,3,4,5,...|
|  7.887786536531237|(10,[0,1,2,3,4,5,...|
| 14.323146365332388|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-0.8995693247765151|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|  5.601801561245534|(10,[0,1,2,3,4,5,...|
|-3.2256352187273354|(10,[0,1,2,3,4,5,...|
| 1.5299675726687754|(10,[0,1,2,3,4,5,...|
| -0.250102447941961|(10,[0,1,2,3,4,5,...|
+----------

In [4]:
lr = LinearRegression(featuresCol='features', labelCol='label', 
                     predictionCol='prediction')

In [5]:
lrModel = lr.fit(traning)

22/09/08 13:24:07 WARN Instrumentation: [52f00697] regParam is zero, which might cause numerical instability and overfitting.
22/09/08 13:24:08 WARN InstanceBuilder$NativeLAPACK: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [11]:
lrModel.coefficients

DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956])

In [13]:
lrModel.intercept

0.14228558260358093

In [15]:
traning_summary = lrModel.summary

In [16]:
traning_summary.r2

0.027839179518600154

In [17]:
traning_summary.rootMeanSquaredError

10.16309157133015

In [6]:
all_data = spark.read.format('libsvm').load('sample_linear_regression_data.txt')

22/09/08 13:25:25 WARN LibSVMFileFormat: 'numFeatures' option not specified, determining the number of features by going though the input. If you know the number in advance, please specify it via 'numFeatures' option to avoid the extra scan.


In [8]:
train_data, test_data = all_data.randomSplit([0.7,0.3])

In [9]:
train_data.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -23.51088409032297|(10,[0,1,2,3,4,5,...|
|-21.432387764165806|(10,[0,1,2,3,4,5,...|
|-20.212077258958672|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-19.872991038068406|(10,[0,1,2,3,4,5,...|
|-19.402336030214553|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|-18.845922472898582|(10,[0,1,2,3,4,5,...|
|-17.803626188664516|(10,[0,1,2,3,4,5,...|
|-17.494200356883344|(10,[0,1,2,3,4,5,...|
|-17.428674570939506|(10,[0,1,2,3,4,5,...|
| -17.32672073267595|(10,[0,1,2,3,4,5,...|
|-17.026492264209548|(10,[0,1,2,3,4,5,...|
| -16.71909683360509|(10,[0,1,2,3,4,5,...|
|-16.692207021311106|(10,[0,1,2,3,4,5,...|
| -16.26143027545273|(10,[0,1,2,3,4,5,...|
|-15.780685032623301|(10,[0,1,2,3,4,5,...|
|-15.437384793431217|(10,[0,1,2,3,4,5,...|
|-15.375857723312297|(10,[0,1,2,3,4,5,...|
|-15.359544879832677|(10,[0,1,2,3,4,5,...|
+----------

In [10]:
train_data.describe().show()

+-------+------------------+
|summary|             label|
+-------+------------------+
|  count|               351|
|   mean|0.3381130139537031|
| stddev| 9.854579112377605|
|    min|-23.51088409032297|
|    max| 27.78383192005107|
+-------+------------------+



In [11]:
test_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                150|
|   mean|0.06682421347960166|
| stddev| 11.361118125799894|
|    min|-28.571478869743427|
|    max| 27.111027963108548|
+-------+-------------------+



In [12]:
correct_model = lr.fit(train_data)

22/09/08 13:25:40 WARN Instrumentation: [11fb4daf] regParam is zero, which might cause numerical instability and overfitting.


In [13]:
test_results = correct_model.evaluate(test_data)

In [14]:
test_results.rootMeanSquaredError

11.401233691872427

In [15]:
unlabeled_data = test_data.select('features')

In [16]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 20 rows



In [17]:
predictions = correct_model.transform(unlabeled_data)

In [18]:
predictions.show()

+--------------------+-------------------+
|            features|         prediction|
+--------------------+-------------------+
|(10,[0,1,2,3,4,5,...|  0.348915716276443|
|(10,[0,1,2,3,4,5,...| 0.4145330770738066|
|(10,[0,1,2,3,4,5,...| 1.5530960401611442|
|(10,[0,1,2,3,4,5,...|-2.3726359610536667|
|(10,[0,1,2,3,4,5,...| 0.5810524867080473|
|(10,[0,1,2,3,4,5,...| 1.8806501191361398|
|(10,[0,1,2,3,4,5,...|-1.6612167211317903|
|(10,[0,1,2,3,4,5,...|   1.40411855870308|
|(10,[0,1,2,3,4,5,...|  1.845157126987007|
|(10,[0,1,2,3,4,5,...|  1.285830463885372|
|(10,[0,1,2,3,4,5,...| 1.6814290476667935|
|(10,[0,1,2,3,4,5,...| 0.6057776362180677|
|(10,[0,1,2,3,4,5,...| 0.7351311213362951|
|(10,[0,1,2,3,4,5,...|0.08505530407062264|
|(10,[0,1,2,3,4,5,...|-1.3242795281603914|
|(10,[0,1,2,3,4,5,...| 1.1018643132200387|
|(10,[0,1,2,3,4,5,...| 1.8311189178270237|
|(10,[0,1,2,3,4,5,...|  2.493950267770243|
|(10,[0,1,2,3,4,5,...| 1.9327363113105753|
|(10,[0,1,2,3,4,5,...| -3.810273582711204|
+----------